<h2>Podcast Downloader</h2>

This code scrapes podcast data from Google Podcasts, downloads the podcast and transcribes it

In [28]:
# Import the required libraries
import requests
import time
import os
import re
from bs4 import BeautifulSoup

In [2]:
# Link to the rss feed of a podcast 
rss_feed_url = "https://www.omnycontent.com/d/playlist/4bb33704-615b-4054-aae9-ace500fd4197/c6e18bb1-650c-43bf-a852-ace600482419/9bac1d6d-3f3a-428a-b8d1-ace600491b2b/podcast.rss"
page = requests.get(rss_feed_url)
soup = BeautifulSoup(page.content, 'xml')

In [3]:
# Find all podcasts from the RSS Feed
podcast_items = soup.find_all('item')

In [10]:
# Download the podcast in mp3 format and save it to a local folder
count = 0
for podcast in podcast_items:
    if count == 5:
        break
    else:
        mp3_url = podcast.find('enclosure')['url']
        mp3_file = requests.get(mp3_url)
        title = podcast.find('title').text
        title = re.sub(r'[%/&!@#\*\$\?\+\^\\.\\\\]', '', title)[:100]
        with open(f'./downloads/{title}.mp3', 'wb') as f:
            f.write(mp3_file.content)
        count += 1

<h4>Transcribe Podcasts</h4>

We will use Assembly AI to transcribe our podcasts

In [ ]:
#Setup the API key and grab it from our Environment Variables
api_key=os.environ["ASSEMBLY_AI_KEY"]

In [21]:
#Upload the podcasts to Assembly AI
headers = {"authorization": api_key, "content-type": "application/json" }
file_names = os.listdir('./downloads')
output_ids = []

def read_file(filename, chunk_size=5242880):
    with open(filename, 'rb') as _file:
        while True:
            data = _file.read(chunk_size)
            if not data:
                break
            yield data
for file in file_names:
    file_path = f'./downloads/{file}'
    print("Uploading", file)
    response = requests.post('https://api.assemblyai.com/v2/upload', headers=headers, data=read_file(file_path))
    
    endpoint = "https://api.assemblyai.com/v2/transcript"
    json = {
        "audio_url": response.json()['upload_url'],
        "audio_start_from": 300000,
        "audio_end_at": 600000   
    }
    response = requests.post(endpoint, json=json, headers=headers)
    output_ids.append(response.json()['id'])

Uploading 10 Money Rules.mp3
Uploading Our relationship with our parents.mp3
Uploading Should you be nice or honest.mp3
Uploading Who hurt you.mp3
Uploading Your best friend.mp3


In [29]:
#Grab transcript of the uploaded podcasts and save as .txt to the Transcripts folder
i=0
for transcript_id in output_ids:
    endpoint = f'https://api.assemblyai.com/v2/transcript/{transcript_id}'
    while True:
        response = requests.get(endpoint,headers=headers)
        status = response.json()['status']
        if status == 'completed':
            print("Success!, Got transcript")
            with open(f'./transcripts/{file_names[i]}.txt','w') as f:
                f.write(response.json()['text'])
                i+=1
            break
        print("Couldn't retrieve transcript, try again in 3 minutes")
        time.sleep(180)

Got transcript
Got transcript
Got transcript
Got transcript
Got transcript
